In [1]:
from scipy.optimize import Bounds, minimize
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from numpy.linalg import norm
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

### Implementacja

In [2]:
class FuzzySVM():
    def __init__(self, kernel='linear', C=1, gamma=1):
        self.C = C
        self.kernel = kernel
        
        if kernel == 'linear':
            self.kernel_function = np.dot
        elif kernel == 'radial':
            self.kernel_function = lambda x, y: np.exp(-gamma*np.sum((x-y)**2))
  
        
    def fit(self, x, y, s):
        self.train_x = x
        self.train_y = y
        
        initial_alfa = [si*self.C/2 for si in s]
        upper_bounds = [si*self.C for si in s]
        alfa_bounds = Bounds(np.zeros(len(x)), upper_bounds)
        
        n_samples = len(x)
        kernel_matrix = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                kernel_matrix[i,j] = self.kernel_function(x[i], x[j])
        
        res = minimize(FuzzySVM.objective_function, initial_alfa, args=(kernel_matrix, y), bounds=alfa_bounds, method='L-BFGS-B')
        
        self.support_vectors = x[res.x>1e-4]
        self.support_labels = y[res.x>1e-4]
        self.lagrange_coef = res.x[res.x > 1e-4]        

        n_support_vectors = len(self.support_vectors)
        
        support_matrix = np.zeros((n_support_vectors, n_support_vectors))
        for i in range(n_support_vectors):
            for j in range(len(self.support_vectors)):
                support_matrix[i,j] = self.kernel_function(self.support_vectors[i], self.support_vectors[j])
        

        
        b = np.mean(self.support_labels - np.sum(self.lagrange_coef[:, None] * self.support_labels[:, None] * support_matrix, axis=0))
        self.b = b
        
        if self.kernel == 'linear':
            self.w = np.sum(self.lagrange_coef[:, None] * self.support_labels[:, None] * self.support_vectors, axis=0)
        else:
            self.w = 0
        
    def objective_function(lam, kernel_matrix, y):
        lam_y = lam * y
        quadratic_term = 0.5 * np.dot(lam_y, np.dot(kernel_matrix, lam_y))
        return -np.sum(lam) + quadratic_term
            
    def predict(self, x):
        results = []
        for sample in x:
            result = self.b + np.sum(self.lagrange_coef * self.support_labels * np.array([self.kernel_function(sv, sample) for sv in self.support_vectors]))
            results.append(1 if result > 0 else -1)
        return np.array(results)  

### Przynależność do klas w zależności od odległości od klastra

In [3]:
class DistanceBasedMembership():
    def __init__(self):
        self.kmeans = KMeans(n_clusters=2, random_state=1)
        self.scaler = MinMaxScaler(clip=True)
        
    def fit_transform(self, x):
        scaled_data = self.scaler.fit_transform(x)
        self.kmeans.fit(scaled_data)
        center = self.kmeans.cluster_centers_[0]
        
        distances = np.array([])
        for element in x:
            distances = np.append(distances, norm(element-center))
            
        self.max_dist = max(distances)
        distances = distances/self.max_dist
        
        return scaled_data, distances
    
    def transform(self, x):
        scaled_data = self.scaler.transform(x)
        center = self.kmeans.cluster_centers_[0]
        
        distances = np.array([])
        for element in x:
            distances = np.append(distances, norm(element-center))
            
        distances = distances/self.max_dist
        
        return scaled_data, distances

### Porównanie działania Fuzzy SVM ze zwykłym SVM na zbiorze iris z dwoma klasami

$\mu_i = 1=\frac{d(x_i, center_i)}{max_j d(x_j, center_j)}$

In [4]:
df = pd.read_csv("hf://datasets/scikit-learn/iris/Iris.csv")

df_dataset = df[(df['Species']=='Iris-versicolor') | (df['Species']=='Iris-virginica')]
df_dataset = df_dataset[['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm', 'Species']]
df_dataset['Species'] = df_dataset['Species'].map({'Iris-versicolor':1, 'Iris-virginica':-1})
x_train, x_test, y_train, y_test = train_test_split(df_dataset[['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']].to_numpy(), df_dataset['Species'].to_numpy(), random_state=1)

In [5]:
membership_fun = DistanceBasedMembership()
scaled_train, membership_train = membership_fun.fit_transform(x_train)
scaled_test, membership_test = membership_fun.transform(x_test)

### Fuzzy SVM

In [6]:
fsvm = FuzzySVM(C=1)
fsvm.fit(scaled_train, y_train, membership_train)
predictions = fsvm.predict(scaled_test)
accuracy_score(y_test, predictions)

0.92

### Zwykły SVM z sklearn

In [7]:
svm = SVC(kernel='linear', C=1)
svm.fit(scaled_train, y_train)

SVC(C=1, kernel='linear')

In [8]:
preds = svm.predict(scaled_test)
accuracy_score(y_test, preds)

0.88

### Fuzzy SVM z radial kernel

In [11]:
fsvm = FuzzySVM(C=1, kernel='radial')
fsvm.fit(scaled_train, y_train, membership_train)
predictions = fsvm.predict(scaled_test)
accuracy_score(y_test, predictions)

0.88

### Zwykły SVM z radial kernel

In [12]:
svm = SVC(kernel='rbf', C=1)
svm.fit(scaled_train, y_train)
preds = svm.predict(scaled_test)
accuracy_score(y_test, preds)

0.84